In [2]:
from seq_net import *
net = SeqNet()

In [3]:
import os
import pickle

data = {}
for f in os.listdir('data'):
    data[f[:-2]] = pickle.load(open('data/' + f, 'rb'), encoding='latin1')
    
labels = [x for x in data]
testing = ['NF2']
training = [x for x in labels if x != 'NF2']

val = []
tran = []

In [ ]:
for x in data:
    print(x, len(data[x]))

In [ ]:
from tqdm import trange
import random 

for i in trange(100):
    gene = random.choice(training)
    samples = [(dna_vec(a), b) for a, b in data[gene]]
    tran.append(net.evaluate(samples, 10))
    net.train(samples, 10)
    validation = data[testing[0]]
    samples = [(dna_vec(a), b) for a, b in validation]
    if i % 2:
        val.append(net.evaluate(samples, 10))
    else:
        val.append(val[-1])

In [ ]:
pickle.dump(val, open('results/validation.p', 'wb'))
pickle.dump(tran, open('results/training.p', 'wb'))

In [ ]:
saver = tf.train.Saver()
saver.save(net.sess, 'results/model.ckpt')

In [ ]:
import matplotlib.pyplot as plt

smooth = 5
plt.title('Validation Reward')
plt.xlabel('Iteration')
plt.plot(np.convolve(val, [1 / smooth] * smooth)[smooth:-smooth])
plt.savefig('results/validation.png')
plt.show()

smooth = 20
plt.title('Training Reward')
plt.xlabel('Iteration')
plt.plot(np.convolve(tran, [1 / smooth] * smooth)[smooth:-smooth])
plt.savefig('results/training.png')
plt.show()

In [6]:
test_data = data[training[3]]

s, a, r = net.path([(dna_vec(a), b) for a, b in test_data])
top3 = sorted(test_data, key=lambda x: x[1])[-3:]
print(top3)

print()
print('best seen\t\t', 'rate\t\t', 'action\t\t\t', 'reward\t\t')
for state, action, reward in zip(s, a, r):
    best_seen = max(state, key=lambda x: x[1])
    print(vec_dna(best_seen[0]), '\t', best_seen[1], '\t', vec_dna(action), '\t', reward)

[('GGCCAAGAAGAACTTAAGCG', 0.768913059), ('GCTGGATTCTGACCAGCACA', 0.795121081), ('GCAAAACCTGGAAAACCCAA', 0.95195016)]

best seen		 rate		 action			 reward		
GCAGCAGGCTGCACTCCTGC 	 0.36806574 	 GTAAGAAGACTCTTCGAATT 	 0.09874929645
GCAGCAGGCTGCACTCCTGC 	 0.36806574 	 GTCCAGCGACCTGGTTAGTC 	 0.175798198
GCAGCAGGCTGCACTCCTGC 	 0.36806574 	 TGACGACTCAGTGGTGGCGT 	 0.073294316
GCAGCAGGCTGCACTCCTGC 	 0.36806574 	 GACAGTATGAAGAAAACCCG 	 0.10119156245000001
GCAGCAGGCTGCACTCCTGC 	 0.36806574 	 AAGTATGGAGGGGGTCGGCT 	 0.16918857719999997
GAGCATAGACAGTATGTGTA 	 0.375974616 	 GTAAGCCTTTTCAAATAGGG 	 0.06698301165
GAGCATAGACAGTATGTGTA 	 0.375974616 	 GCGCTGGTCCTTTCGTTAGG 	 0.13773943349999998
GAGCATAGACAGTATGTGTA 	 0.375974616 	 GCTAGACCACATTCAGAGTG 	 0.0862152755
GAGCATAGACAGTATGTGTA 	 0.375974616 	 GCATAAGAAGCGAGTTGATG 	 0.080707042
GAGCATAGACAGTATGTGTA 	 0.375974616 	 GTGTTCCTGTCACCTATTGG 	 0.037300085999999996
GAGCATAGACAGTATGTGTA 	 0.375974616 	 CCCGCGCTAAAGGTCATCTG 	 0.34626371780000004
ACTGGGCTAAC